In [0]:


import sys
import json
import os

notebook_path = os.getcwd()
sys.path.insert(0, notebook_path+'/self_amplify')


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import pandas as pd
import self_amplify
from self_amplify import CustomWrapper, self_amplifier

In [0]:
def load_model(model_name, access_token):
    n_gpus = torch.cuda.device_count()
#     max_memory = "10000MB"
    max_memory = "80GB"
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto", # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
        token = access_token
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name , token = access_token)

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(torch.cuda.device_count())

1


In [0]:
model_name = 'google/gemma-2b-it'
access_token = 'your_HF_access_token
model, tokenizer = load_model(model_name, access_token)
model.to(device)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Out[5]: GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0): GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          

In [0]:
#Loading data
arc_df_train = pd.read_csv('/dbfs/mnt/ekixai/main/data/MiB/phd/self_amplify/ARC-Challenge-Train.csv')
arc_df_train=arc_df_train[arc_df_train.AnswerKey.isin(["A", "B", "C", "D"])]
arc_df_test = pd.read_csv('/dbfs/mnt/ekixai/main/data/MiB/phd/self_amplify/commonsenseqa_data_train.csv', sep=";")
arc_df_test=arc_df_test[arc_df_test.AnswerKey.isin(["A", "B", "C", "D"])]

#instantiate the amplifier
amplifier = self_amplifier(model, tokenizer, device)

In [0]:
prompt = "This is a big test to simplify the xai function"
template, idx = amplifier.preprocess(text=prompt, with_bracket=False)
print(template,idx)
print(amplifier.get_index_min_max(idx))

<bos><start_of_turn>user
This is a big test to simplify the xai function<end_of_turn>
 tensor([[    2,   106,  1645,   108,  1596,   603,   476,  2824,  2121,   577,
         48180,   573,  1141,  1515,  1411,   107,   108]])
(4, 14)


In [0]:
amplifier.generate_with_explanation(model, idx, explainer='LayerDeepLift',
                                          topk_words=4, target=None, layer=None, split_dict=None,
                                          max_new_tokens=1, temperature=1.0, top_k=None)

Out[8]: ['This', 'test', 'the', 'xai']

In [0]:
idx_list_fs = amplifier.generate_context_idx(model, arc_df_train, nb_shot=4, selection_strategy="success")

amplifier.generate_in_context_preprompt(model=model, df=arc_df_train, explainer='LayerDeepLift', topk_words=4, 
                                  idx_list_fs=idx_list_fs, n_steps=3, split_dict=None,
                                  target_map_dico=None)

C
B
A
A
Out[9]: [{'role': 'user',
  'content': "You are presented with multiple choice question, where choices will look like (A) answer1 (B) answer2 (C) answer3 (D) answer4\n    generate 4 keywords providing hints and generate the right single answer\n    Ouput example: The 4 keywords 'word1', 'word2', 'word3', and 'word4' are important to predict that the answer is (A)<eos>\nBees use pollen from flowers to make honey. Bees help flowers by (A) providing them with fertilizer. (B) helping them grow taller. (C) aiding them in reproduction. (D) increasing photosynthesis."},
 {'role': 'assistant',
  'content': "The 4 keywords 'honey', 'help', 'flowers', 'reproduction',  suggest that the answer is (C)<eos>"},
 {'role': 'user',
  'content': 'The marsh willow herb is a plant native to the northeastern United States. It grows best in damp habitats. Which of the following environmental changes would most likely cause a decrease in the marsh willow herb population in an area? (A) a rainstorm las

In [0]:
amplifier.evaluate_fs_with_exp(df_train=arc_df_train, df_test=arc_df_test, model=model, max_new_tokens=50, explainer='LayerDeepLift', idx_list_fs=idx_list_fs,
                          topk_words=5, split_dict=None,
                          n_steps=3, target_map_dico=None, tokenizer_proxy=None, model_proxy=None)

C
B
A
A
0
The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? (A) ignore (B) enforce (C) authoritarian (D) yell at (E) avoid
------------------------------
expected anwer: A
LLM answer LayerDeepLift: 
--------> "school", "efforts", 'punishing', "change", and "avoid", suggest  that the correct answer would be B. The word "enforce" means to give a law or regulation a legal force.
-------------------------------
1
Sammy wanted to go to where the people were.  Where might he go? (A) race track (B) populated areas (C) the desert (D) apartment (E) roadblock
------------------------------
expected anwer: B
LLM answer LayerDeepLift: 
--------> "race track", "populated areas", and "the desert" suggest  that the correct answer would be ( B) Populated areas.
-------------------------------
2
To locate a choker not located in a jewelry box or boutique where would you go? (A) jewelry store (B) neck (C) jewlery box (D) jewelry bo

answer:  " food item


In [0]:
messages

Out[29]: [{'role': 'user',
  'content': "You are presented with several parts of speech.\nSummarise what these parts of speech have in common in a very concise way using as few words as possible\nExample\n['piano', 'guitar', 'saxophone', 'violin', 'cheyenne', 'drum']"},
 {'role': 'assistant', 'content': 'Summarization: musical instrument<eos>'},
 {'role': 'user',
  'content': "['football', 'basketball', 'baseball', 'tennis', 'badmington', 'soccer']"},
 {'role': 'assistant', 'content': 'Summarization: sport<eos>'},
 {'role': 'user',
  'content': "['lion', 'tiger', 'cat', 'pumas', 'panther', 'leopard']"},
 {'role': 'assistant', 'content': 'Summarization: feline-type animal<eos>'},
 {'role': 'user',
  'content': "['far west', 'cowboy', 'cheyenne', 'gun', 'sherif', 'indians']"},
 {'role': 'assistant', 'content': 'Summarization: '}]

In [0]:
#to do : affiner les answer keys pour les enlever des paramètres, en étant dans l'ordre alphabétique

In [0]:
#Fonctions that did not depend on the above class
def get_topk_words(attribution, topk, window=(0,0)):
        topk_indx = np.sort(np.argpartition(attribution[2], -topk)[-topk:])
        top_attribution = [attribution[1][i].replace('"\"',"") for i in topk_indx.tolist()]
        return(top_attribution)

In [0]:
amplifier = self_amplify(model, tokenizer, device)

(4, 14)
<bos><start_of_turn>user
This is a big test to simplify the xai function<end_of_turn>
<start_of_turn>model
The answer is (


<command-2436442466285397>:256: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  final_attributions = torch.tensor(final_attributions / final_attributions.sum())
Out[13]: ['This', 'test', 'simplify', 'xai']

In [0]:
arc_df_train = pd.read_csv('/dbfs/mnt/ekixai/main/data/MiB/phd/self_amplify/ARC-Challenge-Train.csv')
arc_df_train=arc_df_train[arc_df_train.AnswerKey.isin(["A", "B", "C", "D"])]
arc_df_test = pd.read_csv('/dbfs/mnt/ekixai/main/data/MiB/phd/self_amplify/commonsenseqa_data_train.csv', sep=";")
arc_df_test=arc_df_test[arc_df_test.AnswerKey.isin(["A", "B", "C", "D"])]

causal_judgment_train = pd.read_csv('/dbfs/mnt/ekixai/main/data/MiB/phd/self_amplify/causal_judgment_train.csv', sep=";")
causal_judgment_train['len_tokenized'] = causal_judgment_train['question'].apply(lambda t : len(tokenizer.encode(t)))
causal_judgment_train = causal_judgment_train[causal_judgment_train['len_tokenized']<512]

causal_judgment_test = pd.read_csv('/dbfs/mnt/ekixai/main/data/MiB/phd/self_amplify/causal_judgment_test.csv', sep=";")
causal_judgment_test['len_tokenized'] = causal_judgment_test['question'].apply(lambda t : len(tokenizer.encode(t)))
causal_judgment_test = causal_judgment_test[causal_judgment_test['len_tokenized']<512]

In [0]:
amplifier.generate_context_idx(model, arc_df_train, nb_shot=4, selection_strategy="success", answer_keys=['A', 'B', 'C', 'D'])

Out[19]: [195, 313, 758, 963]

In [0]:
idx_list_fs = amplifier.generate_context_idx(model, arc_df_train, nb_shot=4, selection_strategy="success", answer_keys=['A', 'B', 'C', 'D'])

amplifier.generate_in_context_preprompt(model=model, df=arc_df_train, explainer='LayerDeepLift', topk_words=4, 
                                  idx_list_fs=idx_list_fs, answer_keys=['A', 'B', 'C', 'D'], n_steps=3, split_dict=None,
                                  target_map_dico=None)

C
<command-2436442466285397>:256: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  final_attributions = torch.tensor(final_attributions / final_attributions.sum())
C
A
C
Out[16]: [{'role': 'user',
  'content': "You are presented with multiple choice question, where choices will look like (A) answer1 (B) answer2 (C) answer3 (D) answer4\n    generate 4 keywords providing hints and generate the right single answer\n    Ouput example: The 4 keywords 'word1', 'word2', 'word3', and 'word4' are important to predict that the answer is (A)<eos>\nA student accidentally drops a test tube that breaks when it hits the floor. Which method is the best way to retrieve the broken glass? (A) pick up the pieces with a paper towel (B) use a chemical spill kit (C) use a dustpan and broom (D) pick up the pieces by hand"},
 {'role': 'assistant',
  'content': "Th

In [0]:
idx_list_fs = amplifier.generate_context_idx(model, arc_df_train, nb_shot=4, selection_strategy="success", answer_keys=['A', 'B', 'C', 'D'])

amplifier.generate_in_context_preprompt(model=model, df=arc_df_train, explainer='self_topk', topk_words=4, 
                                  idx_list_fs=idx_list_fs, answer_keys=['A', 'B', 'C', 'D'], n_steps=3, split_dict=None,
                                  target_map_dico=None)

Out[17]: [{'role': 'user',
  'content': "You are presented with multiple choice question, where choices will look like (A) answer1 (B) answer2 (C) answer3 (D) answer4\n    generate 4 keywords providing hints and generate the right single answer\n    Ouput example: The 4 keywords 'word1', 'word2', 'word3', and 'word4' are important to predict that the answer is (A)<eos>\nWhich example describes a behavioral adaptation? (A) A bird builds its nest in the ash near a volcano. (B) A whale has the ability to hold its breath for 20 minutes. (C) A fox's hair is white in the winter and brown in the summer. (D) A monkey has long arms that allow it to swing from one branch to another."},
 {'role': 'assistant',
  'content': '4 important keywords to make the prediction are "nest", “ash”, "volcano”, and “bird”.. Therefore the answer is (A)<eos>'},
 {'role': 'user',
  'content': 'Water evaporates and falls back to Earth as rain or snow. What is the primary energy source that drives this cycle? (A) The

In [0]:
idx_list_fs = amplifier.generate_context_idx(model, arc_df_train, nb_shot=4, selection_strategy="success", answer_keys=['A', 'B', 'C', 'D'])


amplifier.evaluate_fs_with_exp(df_train=arc_df_train, df_test=arc_df_test, model=model, max_new_tokens=50, explainer='self_topk', idx_list_fs=idx_list_fs,
                          topk_words=5, answer_keys=['A', 'B', 'C', 'D'], split_dict=None,
                          n_steps=3, target_map_dico=None, tokenizer_proxy=None, model_proxy=None)

0
The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? (A) ignore (B) enforce (C) authoritarian (D) yell at (E) avoid
------------------------------
expected anwer: A
LLM answer self_topk: 
--------> "punishing", "efforts", and "change" are key to understanding the context of this question and choosing the correct answer.
-------------------------------
1
Sammy wanted to go to where the people were.  Where might he go? (A) race track (B) populated areas (C) the desert (D) apartment (E) roadblock
------------------------------
expected anwer: B
LLM answer self_topk: 
--------> "race track", "populated areas", and "the desert" are all places where people live. Sammy would likely go somewhere in a populated area, not in the middle of nowhere. Therefore, the correct answer would be ( B) Populated areas.
-------------------------------
2
To locate a choker not located in a jewelry box or boutique where would you go? (A) j

In [0]:
concepts_list_1 = ["piano", "guitar", "saxophone", "violin", "cheyenne", "drum"]
macro_concept_1 = "musical instrument"

concepts_list_2 = ["football", "basketball", "baseball", "tennis", "badmington", "soccer"]
macro_concept_2 = "sport"

concepts_list_3 = ["lion", "tiger", "cat", "pumas", "panther", "leopard"]
macro_concept_3 = "feline-type animal"

# concepts_list_test = ["hamburger", "hot dog", "sandwich", "cheeseburger", "bigmac", "fries"]

#ok
concepts_list_test = ["American Crocodile", "Killer Bigfoot", "Bigfoot Bonanza", "The BLOODY APE", "Tiger Shark", "Montana legend of mutant/cannibal creatures", "Bigfoot"]

#pas ok
concepts_list_test = ["Monster School as a safe haven", "The Slender Man Mythos", "The Shadow Man", "Monster Hunt", "Monster", "Monster and its intentions", "Thed Ude", "Living monster made of wood", "The Dragon and his activities", "The Man", "Human monsters and the almighty dollar", "Monster Attacks", "Monsters", "The Science Behind the Monster", "Greek mythological monster attack", "Frankenstein legend"]
 
#pas ok
concepts_list_test = ["Deadly witches", "Clairvoyance and demonic possession", "The Spirits and their Influence", "Witchcraft series", "Witch Trials", "Evil Spirit,Possession", "Mediums and spiritual phenomena", "Demonic Influence", "Voodoo", "Witch's Curse", "Evil spirit and manifestation", "Demonic Possession", "The monstrous, vengeful boogie woman", "Spirit Possession and Family Conflict", "Spirit Possession and Serial Killers", "Louis and the witch's spell", "Witch Execution", "The effects of the curse", "Culinary Magic", "Murder and Evil Spirits", "The curse of Bloody Mary", "Magic and power", "Magic paw with magical powers", "Return of Powerful Witches", "Witchcraft and demonic possession", "Witchcraft", "Coven of witches", "Possession and Evil", "The ageing Countess", "Magical Warlock", "Spirit possession", "Possessed Inmates", "Real-life occult practices", "The Magificent Madame Mortem", "Magic trick with needle, floss, and pierced ear", "EVP and demonic forces", "Voodoo and demonic possession", "Season of the Witch", "Ancient Magic", "Witch's potion", "Witch hunt", "The Usher family curse", "Indian sorcerers", "Curse of the SUCCUBUS!", "Magic", "The Curse of Usher", "Curse and spirits", "Magic and illusion", "Witchcraft and necromancy", "Witchcraft and Voodoo", "Spirit Possession", "Personal Demons", "Possessed by an evil spirit", "Magic and sorcery", "Seances", "Witch named Leah Smock", "Voodoo and the Walking Dead", "Witches' Dolls", "Deadly turn", "Ancient Witch", "Adopted daughter becomes possessed by a demonic force", "Possession by spirit", "Curse", "Exorcism gone wrong", "Witchcraft accusation", "Treatment of Citizens and Black Witch", "Demon Possession", "Madame Tirelou and her curse", "Evil Spirits and Occult History"]



concepts_list_test =  ["Gold and Treasure", "Lost gold mine", "The potential exploitation of the valley's silver", "The harshness of the frontier", "Mine Strike", "Banker and secret gold mines", "Gold bullion", "Gold", "California Gold Rush", "Wilderness and Greed", "Mining", "Mining operations", "American claims to the Oregon Territory", "Discovery of gold", "Gold discovery", "The decline of the Wild West", "A.C. Lyles and his westerns", "Silver Mine", "Ferguson's abandoned mine", "Mine Tunnel", "Gold rush", "Gold mine", "Lost Dutchman mine", "Gold-prospecting", "Copperminers", "Mine value", "Gold Creek turmoil"]
 
concepts_list_test =  ["Mining operations", "Discovery of gold", "Gold discovery", "The decline of the Wild West", "A.C. Lyles and his westerns",]

#ok
concepts_list_test = ["Deadly witches", "Clairvoyance and demonic possession", "The Spirits and their Influence", "Witchcraft series", "Witch Trials", "Evil Spirit,Possession", "Mediums and spiritual phenomena", "Demonic Influence", "Voodoo", "Witch's Curse", "Evil spirit and manifestation", "Demonic Possession", "The monstrous, vengeful boogie woman", "Spirit Possession and Family Conflict", "Spirit Possession and Serial Killers", "Louis and the witch's spell"]


preprompt = '''You are presented with several parts of speech.
Summarise what these parts of speech have in common in a very concise way using as few words as possible'''



messages = [{"role": "user", "content": preprompt + "\nExample\n" + str(concepts_list_1)}]
messages.append({"role": "assistant", "content": "Summarization: "+macro_concept_1+"<eos>"})  

messages.append({"role": "user", "content":str(concepts_list_2)})
messages.append({"role": "assistant", "content": "Summarization: "+macro_concept_2+"<eos>"})  

messages.append({"role": "user", "content":str(concepts_list_3)})
messages.append({"role": "assistant", "content": "Summarization: "+macro_concept_3+"<eos>"})  

messages.append({"role": "user", "content":str(concepts_list_test)})
messages.append({"role": "assistant", "content":"Summarization: "})



In [0]:
encoded_input = tokenizer.apply_chat_template(messages, return_tensors='pt')

encoded_input = torch.reshape(encoded_input[0][:-2], (1, encoded_input[0][:-2].shape[0]))

len_input = encoded_input.shape[1]
outputs = model.generate(encoded_input.to(device), max_new_tokens = 50, do_sample=True, num_beams=3, no_repeat_ngram_size=2, early_stopping=True, temperature=0.01)
answer = '"' + tokenizer.decode(outputs[0][len_input:], skip_special_tokens=True)
print("answer: ",answer)

answer:  " supernatural or demonic phenomenon


In [0]:
len(["Deadly witches", "Clairvoyance and demonic possession", "The Spirits and their Influence", "Witchcraft series", "Witch Trials", "Evil Spirit Possession", "Mediums and spiritual phenomena", "Demonic Influence", "Voodoo", "Witch's Curse", "Evil spirit and manifestation", "Demonic Possession", "The monstrous, vengeful boogie woman", "Spirit Possession and Family Conflict", "Spirit Possession and Serial Killers", "Louis and the witch's spell", "Witch Execution", "The effects of the curse", "Culinary Magic", "Murder and Evil Spirits", "The curse of Bloody Mary", "Magic and power", "Magic paw with magical powers", "Return of Powerful Witches", "Witchcraft and demonic possession", "Witchcraft", "Coven of witches", "Possession and Evil", "The ageing Countess", "Magical Warlock", "Spirit possession", "Possessed Inmates", "Real-life occult practices", "The Magificent Madame Mortem", "Magic trick with needle, floss, and pierced ear", "EVP and demonic forces", "Voodoo and demonic possession", "Season of the Witch", "Ancient Magic", "Witch's potion", "Witch hunt", "The Usher family curse", "Indian sorcerers", "Curse of the SUCCUBUS!", "Magic", "The Curse of Usher", "Curse and spirits", "Magic and illusion", "Witchcraft and necromancy", "Witchcraft and Voodoo", "Spirit Possession", "Personal Demons", "Possessed by an evil spirit", "Magic and sorcery", "Seances", "Witch named Leah Smock", "Voodoo and the Walking Dead", "Witches' Dolls", "Deadly turn", "Ancient Witch", "Adopted daughter becomes possessed by a demonic force", "Possession by spirit", "Curse", "Exorcism gone wrong", "Witchcraft accusation", "Treatment of Citizens and Black Witch", "Demon Possession", "Madame Tirelou and her curse", "Evil Spirits and Occult History"]
 )

Out[94]: 69

In [0]:
len(concepts_list_test)

Out[110]: 16